In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

def read_jsonl_file(path):
    json_lines = []
    with open(path, 'r') as f:
        for line in f:
            json_lines.append(json.loads(line.strip()))
    return json_lines

In [2]:
matches = read_jsonl_file('/Users/jooyong/github_locals/CSCI5525_project/team_ids_player_ids.json')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jooyong/github_locals/aml_5525/aml_5525/dataset/processed/team_ids_player_ids.json'

In [8]:
list_all_keys_needed = []

for i in range(len(matches)):
    
    temp = matches[i]['home_players']+matches[i]['away_players']
    
    for k in temp:
        list_all_keys_needed.append(str(k))
        
keys_players = list(set(list_all_keys_needed))

In [9]:
len(keys_players)

1627

In [11]:
selected_attributes_field = ['appearances',
     'losses',
     'wins',
     'draws',
     'mins_played',
     'touches',
     'total_pass',
     'accurate_pass',
     'total_fwd_zone_pass',
     'accurate_fwd_zone_pass',
     'poss_lost_all',
     'poss_lost_ctrl',
     'total_back_zone_pass',
     'total_final_third_passes',
     'accurate_back_zone_pass',
     'attempts_conceded_ibox',
     'successful_final_third_passes',
     'duel_lost',
     'attempts_conceded_obox',
     'duel_won',
     'final_third_entries']


selected_attributes_keeper = ['appearances',
     'losses',
     'wins',
     'mins_played',
     'attempts_conceded_ibox',
     'touches',
     'attempts_conceded_obox',
     'goal_kicks',
     'long_pass_own_to_opp',
     'total_fwd_zone_pass',
     'total_long_balls',
     'total_pass',
     'accurate_pass',
     'poss_lost_all',
     'poss_lost_ctrl',
     'accurate_goal_kicks',
     'accurate_long_balls',
     'ball_recovery',
     'saves',
     'accurate_back_zone_pass',
     'keeper_throws']

In [12]:
with open('/Users/jooyong/github_locals/aml_5525/aml_5525/dataset/merged_data/merged_player_test.json') as f:
  data = json.load(f)

In [13]:
for i in data.keys():
    del(data[i]['seasons'])

In [15]:
def pre_processing_for_average_att(data, role, attributes):
    
    merged_data = copy.deepcopy(data)
    # discard players whose latest season is before 2012-13 (season id:21)
    
    keys = list(merged_data.keys())



    for i in keys:
        
        if i not in keys_players:
            del(merged_data[i])
            
    keys = list(merged_data.keys())

#     for i in keys:
#         temp = list(merged_data[i].keys())
#         for k in temp:
#             if k=='all_season' or k=='fm_data':
#                 temp.remove(k)

#         temp_2 = []
#         for k in temp:
#             if k!='fm_data':
#                 temp_2.append(int(k))

#         if len(temp_2)>0:
#             if np.max(temp_2) < 21:
#                 del(merged_data[i])


    # To calculate mean of each attribute, discard players have empty statistics. 
    

    for i in keys:
        if  len(merged_data[i]['official_stats'])==0:
            del(merged_data[i])
            print('noooo')
        elif len(merged_data[i]['official_stats'])==0:
            del(merged_data[i])
            print('nooo')
    


    keys =list(merged_data.keys())

    for i in keys:

        if 'position' not in list(merged_data[i]['entity']['info'].keys()):
            del(merged_data[i])
    
        if role == 'keeper':
            if merged_data[i]['entity']['info']['position']!='G':
                del(merged_data[i])
                
        if role == 'field':
            if merged_data[i]['entity']['info']['position']=='G':
                del(merged_data[i])

                
                
    keys = list(merged_data.keys())
    
    
#     for i in keys:

#         keys2 = list(merged_data[i].keys())

#         for k in keys2:

#             if k!='all_season' and k!='fm_data':
#                 del(merged_data[i][k])

                
    temp = ['playerId', 'info', 'nationalTeam', 'birth', 'age', 'name', 'id']

    keys = list(merged_data.keys())
    
    for i in keys:

        age = {}

        for k in temp:

            if k != 'age':

                del(merged_data[i]['entity'][k])


            else:
                age_temp = int(merged_data[i]['entity'][k][:2])

                merged_data[i]['age'] = age_temp

        del(merged_data[i]['entity'])

    
    
    keys = list(merged_data.keys())

    temp = {}

    for i in keys:
        official_dict = {}
        official_dict['official_stats'] = merged_data[i]['official_stats']


        temp[i] = official_dict
        temp[i]['age'] = merged_data[i]['age']/35
        
        if 'fm_data' in merged_data[i].keys():
#             fm_dict = {}
#             fm_dict['fm_data'] = merged_data[i]['fm_data']['attributes']
            temp[i]['fm_data'] = merged_data[i]['fm_data']['attributes']
            temp[i]['fm_data']['Length'] = merged_data[i]['fm_data']['Length']
            temp[i]['fm_data']['Weight'] = merged_data[i]['fm_data']['Weight']
            temp[i]['fm_data']['Sell value'] = merged_data[i]['fm_data']['Sell value']
            temp[i]['fm_data']['Wages'] = merged_data[i]['fm_data']['Wages']
            temp[i]['fm_data']['ability'] = merged_data[i]['fm_data']['ability']
            temp[i]['fm_data']['potential'] = merged_data[i]['fm_data']['potential']


    
# 'Foot': 'Right',
# 'Length': 0.633,
# 'Weight': 0.733,
# 'Sell value': 0.046,
# 'Wages': 0.02,
# 'ability': 0.61,
# 'potential': None,

            
    
    for i in temp.keys():

        keys = list(temp[i]['official_stats'].keys())

        for k in keys:
            
            if role == 'field':
                
                if k not in selected_attributes_field:
                    del(temp[i]['official_stats'][k])
                    
            if role == 'keeper':
                
                if k not in selected_attributes_keeper:
                    del(temp[i]['official_stats'][k])
                
    
    temp_2 = copy.deepcopy(temp)
    
    
    a = attributes
        
    
    list_keys_wrong = []

    for i in temp.keys():

        count = 0 


        for k in temp[i]['official_stats']:

            if k in a:
                count+=1

        if count!=21:
            list_keys_wrong.append(i)
    
    for i in list_keys_wrong:
        del(temp[i])

    ave_att_dict = {}


    for i in a:

        sum_att = 0

        for k in temp.keys():

            sum_att += temp[k]['official_stats'][i]
            
        ave_att_dict[i] = round(sum_att/len(temp), 0)


    return ave_att_dict, temp_2

In [16]:
average_keeper, pre_processed_keeper = pre_processing_for_average_att(data, 'keeper', selected_attributes_keeper)

In [17]:
average_keeper

{'appearances': 103.0,
 'accurate_back_zone_pass': 742.0,
 'accurate_pass': 1176.0,
 'attempts_conceded_ibox': 692.0,
 'attempts_conceded_obox': 490.0,
 'game_started': 89.0,
 'mins_played': 8021.0,
 'poss_lost_all': 1206.0,
 'poss_lost_ctrl': 1202.0,
 'total_back_zone_pass': 880.0,
 'total_long_balls': 1555.0,
 'total_pass': 2165.0,
 'touches': 3507.0,
 'ball_recovery': 807.0,
 'long_pass_own_to_opp': 1585.0,
 'total_fwd_zone_pass': 1286.0,
 'accurate_fwd_zone_pass': 434.0,
 'accurate_long_balls': 588.0,
 'final_third_entries': 672.0,
 'goals_conceded': 121.0,
 'total_final_third_passes': 575.0}

In [19]:
all_keeper = pre_processed_keeper

In [22]:
average_field, pre_processed_field = pre_processing_for_average_att(data, 'field', selected_attributes_field)

In [23]:
average_field

{'appearances': 95.0,
 'accurate_pass': 2331.0,
 'mins_played': 6295.0,
 'poss_lost_all': 974.0,
 'poss_lost_ctrl': 933.0,
 'total_fwd_zone_pass': 1828.0,
 'total_pass': 2893.0,
 'touches': 4241.0,
 'total_back_zone_pass': 1211.0,
 'accurate_back_zone_pass': 1074.0,
 'accurate_fwd_zone_pass': 1290.0,
 'attempts_conceded_ibox': 536.0,
 'total_final_third_passes': 912.0,
 'duel_lost': 373.0,
 'attempts_conceded_obox': 376.0,
 'duel_won': 375.0,
 'successful_final_third_passes': 624.0,
 'fwd_pass': 860.0,
 'final_third_entries': 405.0,
 'ball_recovery': 344.0,
 'leftside_pass': 633.0}

In [25]:
all_field = pre_processed_field

In [28]:
with open('/Users/jooyong/github_locals/aml_5525/aml_5525/dataset/merged_data/merged_player_test.json') as f:
  merged_data = json.load(f)

In [29]:
keys = list(merged_data.keys())
for i in keys:
    temp = list(merged_data[i].keys())
    for k in temp:
        if k=='all_season' or k=='fm_data':
            temp.remove(k)

    temp_2 = []
    for k in temp:
        if k!='fm_data':
            temp_2.append(int(k))

    if len(temp_2)>0:
        if np.max(temp_2) < 21:
            del(merged_data[i])

ValueError: invalid literal for int() with base 10: 'entity'

In [ ]:
keys_empty_stats = []

count = 0
count2 = 0
keys = list(merged_data.keys())
for i in keys:
    if len(merged_data[i].keys())==1 and len(merged_data[i][list(merged_data[i].keys())[0]]['official_stats'])==0:
        count += 1
        keys_empty_stats.append(i)
        #del(merged_data[i])
    elif len(merged_data[i].keys())==2 and len(merged_data[i][list(merged_data[i].keys())[0]]['official_stats'])==0:
        count +=1
        keys_empty_stats.append(i)
        #del(merged_data[i])
    else:
        count2 +=1

In [33]:
keys = list(merged_data.keys())

for i in keys:
    
    if 'position' not in list(merged_data[i]['all_season']['entity']['info'].keys()):
        del(merged_data[i])
    
    if merged_data[i]['all_season']['entity']['info']['position']=='G':
        del(merged_data[i])
        
keys = list(merged_data.keys())
for i in keys:
    
    keys2 = list(merged_data[i].keys())
    
    for k in keys2:
        
        if k!='all_season' and k!='fm_data':
            del(merged_data[i][k])

temp = ['playerId', 'info', 'nationalTeam', 'birth', 'age', 'name', 'id']

keys = list(merged_data.keys())
for i in keys:
    
    age = {}
    
    for k in temp:
        
        if k != 'age':
            del(merged_data[i]['all_season']['entity'][k])
            
        
        else:
            age_temp = int(merged_data[i]['all_season']['entity'][k][:2])
            
            merged_data[i]['all_season']['age'] = age_temp
            
    del(merged_data[i]['all_season']['entity'])
    
keys = list(merged_data.keys())

temp = {}

for i in keys:
    
    
    temp[i] = merged_data[i]['all_season']
    temp[i]['age'] = merged_data[i]['all_season']['age']/35
    
    if 'fm_data' in merged_data[i].keys():
        temp[i]['fm_data'] = merged_data[i]['fm_data']
    
for i in temp.keys():
    
    keys = list(temp[i]['official_stats'].keys())
    
    for k in keys:
        if k not in a:
            del(temp[i]['official_stats'][k])

KeyError: 'entity'

In [129]:
def pre_processing_for_merged_data(data, role):
    
    merged_data = copy.deepcopy(data)
    # discard players whose latest season is before 2012-13 (season id:21)
    
    keys = list(merged_data.keys())



    for i in keys:
        
        if i not in keys_players:
            del(merged_data[i])
            

    

    keys =list(merged_data.keys())

    for i in keys:

        if 'position' not in list(merged_data[i]['entity']['info'].keys()):
            del(merged_data[i])
    
        if role == 'keeper':
            if merged_data[i]['entity']['info']['position']!='G':
                del(merged_data[i])
                
        if role == 'field':
            if merged_data[i]['entity']['info']['position']=='G':
                del(merged_data[i])

                
                
                
    keys = list(merged_data.keys())
    for i in keys:

        keys2 = list(merged_data[i].keys())

        for k in keys2:

            if k!='all_season' and k!='fm_data':
                del(merged_data[i][k])

    temp = ['playerId', 'info', 'nationalTeam', 'birth', 'age', 'name', 'id']
    
    

    keys = list(merged_data.keys())
    
    for i in keys:

        age = {}

        for k in temp:

            if k != 'age':

                del(merged_data[i]['entity'][k])


            else:
                age_temp = int(merged_data[i]['entity'][k][:2])

                merged_data[i]['age'] = age_temp

        del(merged_data[i]['entity'])


        
        
    keys = list(merged_data.keys())

    temp = {}

    for i in keys:
        official_dict = {}
        official_dict['official_stats'] = merged_data[i]['official_stats']


        temp[i] = official_dict
        temp[i]['age'] = merged_data[i]['age']/35
        
        if 'fm_data' in merged_data[i].keys():

            temp[i]['fm_data'] = merged_data[i]['fm_data']['attributes']


            
    
    for i in temp.keys():

        keys = list(temp[i]['official_stats'].keys())

        for k in keys:
            
            if role == 'field':
                
                if k not in selected_attributes_field:
                    del(temp[i]['official_stats'][k])
                    
            if role == 'keeper':
                
                if k not in selected_attributes_keeper:
                    del(temp[i]['official_stats'][k])
                
    
    temp_2 = copy.deepcopy(temp)

    return temp_2

In [132]:
all_field = pre_processing_for_merged_data(merged_data, 'field')

KeyError: 'entity'

In [146]:
len(all_field)

1499

In [151]:
all_field['51423']

{'official_stats': {'accurate_back_zone_pass': 1042.0,
  'accurate_fwd_zone_pass': 1970.0,
  'accurate_pass': 2950.0,
  'appearances': 110.0,
  'attempts_conceded_ibox': 926.0,
  'attempts_conceded_obox': 538.0,
  'draws': 21.0,
  'duel_lost': 600.0,
  'duel_won': 714.0,
  'final_third_entries': 784.0,
  'losses': 53.0,
  'mins_played': 9548.0,
  'poss_lost_all': 1353.0,
  'poss_lost_ctrl': 1353.0,
  'successful_final_third_passes': 1094.0,
  'total_back_zone_pass': 1199.0,
  'total_final_third_passes': 1535.0,
  'total_fwd_zone_pass': 2738.0,
  'total_pass': 3687.0,
  'touches': 5689.0,
  'wins': 36.0},
 'age': 0.8,
 'fm_data': {'Corners': 0.55,
  'Crossing': 0.65,
  'Dribbling': 0.7,
  'Finishing': 0.7,
  'First Touch': 0.65,
  'Free Kick Taking': 0.5,
  'Heading': 0.45,
  'Long Shots': 0.65,
  'Long Throws': 0.45,
  'Marking': 0.5,
  'Passing': 0.7,
  'Penalty Taking': 0.5,
  'Tackling': 0.65,
  'Technique': 0.75,
  'Aggression': 0.7,
  'Anticipation': 0.65,
  'Bravery': 0.65,
  'Co

In [144]:
len(pre_processed_field)

1499

In [226]:
temp = all_field

for i in temp.keys():
#     if 'fm_data' in temp[i].keys():
#         for k in temp[i]['fm_data']['attributes'].keys():
#             k_fixed = k+'_fm'
#             temp[i][k_fixed] = temp[i]['fm_data']['attributes'][k]
        
        for k in temp[i]['fm_data'].keys():
            if k != 'attributes' and k!='age':
                temp[i][k] = temp[i]['fm_data'][k]
        
        del(temp[i]['fm_data'])

KeyError: 'fm_data'

In [272]:
temp = all_field

In [273]:
temp['31442']

{'official_stats': {'accurate_back_zone_pass': 48.0,
  'accurate_fwd_zone_pass': 205.0,
  'accurate_pass': 253.0,
  'appearances': 40.0,
  'attempts_conceded_ibox': 190.0,
  'attempts_conceded_obox': 103.0,
  'draws': 13.0,
  'duel_lost': 296.0,
  'duel_won': 173.0,
  'final_third_entries': 46.0,
  'losses': 13.0,
  'mins_played': 2086.0,
  'poss_lost_all': 334.0,
  'poss_lost_ctrl': 334.0,
  'successful_final_third_passes': 139.0,
  'total_back_zone_pass': 68.0,
  'total_final_third_passes': 232.0,
  'total_fwd_zone_pass': 355.0,
  'total_pass': 415.0,
  'touches': 823.0,
  'wins': 14.0},
 'age': 0.6,
 'fm_data': {'Corners': 0.35,
  'Crossing': 0.4,
  'Dribbling': 0.55,
  'Finishing': 0.6,
  'First Touch': 0.6,
  'Free Kick Taking': 0.35,
  'Heading': 0.6,
  'Long Shots': 0.4,
  'Long Throws': 0.35,
  'Marking': 0.25,
  'Passing': 0.35,
  'Penalty Taking': 0.4,
  'Tackling': 0.25,
  'Technique': 0.55,
  'Aggression': 0.7,
  'Anticipation': 0.55,
  'Bravery': 0.55,
  'Composure': 0.55,

In [274]:
keys_empty_stats = []

count = 0
count2 = 0
keys = list(temp.keys())
for i in keys:
    if len(temp[i]['official_stats'])==0:
        count += 1
        keys_empty_stats.append(i)
        #del(merged_data[i])
    else:
        count2 +=1

In [276]:
len(keys_empty_stats)

0

In [277]:
count

0

In [278]:
keys_empty_stats

[]

In [99]:
set_a = set(keys_empty_stats)

In [279]:
len(temp)

1499

In [101]:
for i in keys_empty_stats:
    
    temp[i]['official_stats'] = average_field

In [280]:
list_keys_wrong = []


for i in temp.keys():

    count = 0 

        
    for k in temp[i]['official_stats']:

        if k in selected_attributes_field:
            count+=1

    if count!=21:
        list_keys_wrong.append(i)

In [283]:
final_field_dict = {}

for i in temp.keys():
    
    final_field_dict[i] = temp[i]

In [284]:
len(final_field_dict)

1499

In [ ]:
with open('/Users/jooyong/github_locals/aml_5525/aml_5525/dataset/merged_data/merged_player.json') as f:
  merged_data = json.load(f)

In [ ]:
keys = list(merged_data.keys())
for i in keys:
    temp = list(merged_data[i].keys())
    for k in temp:
        if k=='all_season' or k=='fm_data':
            temp.remove(k)

    temp_2 = []
    for k in temp:
        if k!='fm_data':
            temp_2.append(int(k))

    if len(temp_2)>0:
        if np.max(temp_2) < 21:
            del(merged_data[i])

In [ ]:
keys = list(merged_data.keys())

for i in keys:
    
    if 'position' not in list(merged_data[i]['all_season']['entity']['info'].keys()):
        del(merged_data[i])
    
    if merged_data[i]['all_season']['entity']['info']['position']!='G':
        del(merged_data[i])
        
keys = list(merged_data.keys())
for i in keys:
    
    keys2 = list(merged_data[i].keys())
    
    for k in keys2:
        
        if k!='all_season' and k!='fm_data':
            del(merged_data[i][k])

temp = ['playerId', 'info', 'nationalTeam', 'birth', 'age', 'name', 'id']

keys = list(merged_data.keys())
for i in keys:
    
    age = {}
    
    for k in temp:
        
        if k != 'age':
            del(merged_data[i]['all_season']['entity'][k])
            
        
        else:
            age_temp = int(merged_data[i]['all_season']['entity'][k][:2])
            
            merged_data[i]['all_season']['age'] = age_temp
            
    del(merged_data[i]['all_season']['entity'])
    
keys = list(merged_data.keys())

temp = {}

for i in keys:
    
    
    temp[i] = merged_data[i]['all_season']
    temp[i]['age'] = merged_data[i]['all_season']['age']/35
    
    if 'fm_data' in merged_data[i].keys():
        print('1')
        temp[i]['fm_data'] = merged_data[i]['fm_data']
    
for i in temp.keys():
    
    keys = list(temp[i]['official_stats'].keys())
    
    for k in keys:
        if k not in a:
            del(temp[i]['official_stats'][k])

In [ ]:
for i in temp.keys():
    
    if 'fm_data' in temp[i].keys():
        for k in temp[i]['fm_data']['attributes'].keys():
            k_fixed = k+'_fm'
            temp[i][k_fixed] = temp[i]['fm_data']['attributes'][k]
        
        for k in temp[i]['fm_data'].keys():
            if k != 'attributes' and k != 'age':
                temp[i][k] = temp[i]['fm_data'][k]
        
        del(temp[i]['fm_data'])

In [285]:
temp = all_keeper

In [286]:
keys_empty_stats = []

count = 0
count2 = 0
keys = list(temp.keys())
for i in keys:
    if len(temp[i]['official_stats'])==0:
        count += 1
        keys_empty_stats.append(i)
        #del(merged_data[i])

    else:
        count2 +=1

In [287]:
keys_empty_stats

[]

In [288]:
for i in keys_empty_stats:
    
    temp[i]['official_stats'] = ave_att_dict

In [289]:
keys_empty_stats

[]

In [290]:
list_keys_wrong = []


for i in temp.keys():

    count = 0 

        
    for k in temp[i]['official_stats']:

        if k in selected_attributes_keeper:
            count+=1

    if count!=21:
        list_keys_wrong.append(i)

In [291]:
list_keys_wrong

['3732',
 '10727',
 '13485',
 '13556',
 '13777',
 '13795',
 '9293',
 '4110',
 '4348',
 '4556',
 '2761',
 '3457',
 '4727',
 '4909',
 '5361',
 '10479',
 '19675',
 '19771',
 '54392']

In [292]:
for i in list_keys_wrong:
    print(i)
    print(len(temp[i]['official_stats'].keys()))

3732
19
10727
20
13485
20
13556
20
13777
20
13795
20
9293
18
4110
20
4348
20
4556
20
2761
19
3457
20
4727
20
4909
20
5361
17
10479
20
19675
20
19771
20
54392
20


In [293]:
len(temp['3732']['official_stats'])

19

In [294]:
final_keeper_dict = {}

for i in temp.keys():
    
    final_keeper_dict[i] = temp[i]

In [295]:
len(final_keeper_dict)

128

In [ ]:
with open('/Users/jooyong/github_locals/aml_5525/aml_5525/dataset/merged_data/merged_player.json') as f:
  merged_data = json.load(f)

In [ ]:
merged_data['4609']

In [ ]:
merged_data['4741']

In [ ]:
# keys = list(temp.keys())

# for i in keys:
    
#     if 'mins_played' in temp[i]['official_stats'].keys():
#         for k in temp[i]['official_stats'].keys():
#             temp[i]['official_stats'][k] = temp[i]['official_stats'][k] / temp[i]['official_stats'] \
#             ['mins_played']
            
#     else:
#         print(temp[i])

In [107]:
for i in final_field_dict.keys():
    final_field_dict[i]['age'] = round(final_field_dict[i]['age'], 3)
    
# for i in final_keeper_dict.keys():
#     final_keeper_dict[i]['age'] = round(final_keeper_dict[i]['age'], 3)

In [195]:
len(final_field_dict)

1499

In [196]:
final_field_dict

{'3102': {'official_stats': {'accurate_back_zone_pass': 5342.0,
   'accurate_fwd_zone_pass': 5640.0,
   'accurate_pass': 10970.0,
   'appearances': 249.0,
   'attempts_conceded_ibox': 1117.0,
   'attempts_conceded_obox': 1014.0,
   'draws': 59.0,
   'duel_lost': 1051.0,
   'duel_won': 1145.0,
   'final_third_entries': 1719.0,
   'losses': 41.0,
   'mins_played': 16228.0,
   'poss_lost_all': 1821.0,
   'poss_lost_ctrl': 1654.0,
   'successful_final_third_passes': 1938.0,
   'total_back_zone_pass': 5707.0,
   'total_final_third_passes': 2539.0,
   'total_fwd_zone_pass': 6599.0,
   'total_pass': 12253.0,
   'touches': 15086.0,
   'wins': 149.0},
  'age': 1.0},
 '3116': {'official_stats': {'accurate_back_zone_pass': 98.0,
   'accurate_fwd_zone_pass': 89.0,
   'accurate_pass': 185.0,
   'appearances': 13.0,
   'attempts_conceded_ibox': 85.0,
   'attempts_conceded_obox': 57.0,
   'draws': 2.0,
   'duel_lost': 32.0,
   'duel_won': 56.0,
   'final_third_entries': 41.0,
   'losses': 8.0,
   'mi

In [191]:
len(final_keeper_dict)

128

In [296]:
final_total_players = dict(final_field_dict, **final_keeper_dict)

In [297]:
len(final_total_players)

1627

In [298]:
with open('merged_player_after_pre_processing.json', 'w', encoding='utf-8') as make_file:
    json.dump(final_total_players, make_file, ensure_ascii=False, indent='\t')

In [ ]:

for i in final_total_players.keys():
    
    if 'mins_played' not in final_total_players[i]['official_stats']:
        print(i)
        print('yes')